### Ingest data on Google Earth Engine (WRI bucket)

* Purpose of script: This notebook will upload the geotiff files from the Google Cloud Storage to the WRI/aqueduct earthengine bucket. 
* Author: Rutger Hofste
* Kernel used: python36
* Date created: 20170802

## Preparation

* Authorize earthengine by running in your terminal: `earthengine authenticate`  
* you need to have access to the WRI-Aquaduct (yep a Google employee made a typo) bucket to ingest the data. Rutger can grant access to write to this folder. 
* Have access to the Google Cloud Storage Bucker

Run in your terminal `gcloud config set project aqueduct30`

In [1]:
import subprocess
import datetime
import os
import time
from datetime import timedelta
import re
import pandas as pd

## Settings

In [2]:
GCS_BASE = "gs://aqueduct30_v01/Y2017M08D02_RH_Upload_to_GoogleCS_V01/"

In [3]:
EE_BASE = "projects/WRI-Aquaduct/PCRGlobWB20V05"

## Functions

In [4]:
def splitKey(key):
    # will yield the root file code and extension of a set of keys
    prefix, extension = key.split(".")
    fileName = prefix.split("/")[-1]
    parameter = fileName[:-12]
    month = fileName[-2:] #can also do this with regular expressions if you like
    year = fileName[-7:-3]
    identifier = fileName[-11:-8]
    outDict = {"fileName":fileName,"extension":extension,"parameter":parameter,"month":month,"year":year,"identifier":identifier}
    return outDict

def splitParameter(parameter):
    #values = parameter.split("_")
    values = re.split("_|-", parameter) #soilmoisture uses a hyphen instead of underscore between the years
    keys = ["geographic_range","temporal_range","indicator","temporal_resolution","units","spatial_resolution","temporal_range_min","temporal_range_max"]
    # ['global', 'historical', 'PDomWN', 'month', 'millionm3', '5min', '1960', '2014']
    outDict = dict(zip(keys, values))
    outDict["parameter"] = parameter
    return outDict


## Script

In [5]:
command = ("earthengine create folder %s") %EE_BASE

In [6]:
print(command)

earthengine create folder projects/WRI-Aquaduct/PCRGlobWB20V05


In [7]:
subprocess.check_output(command,shell=True)

'Asset projects/WRI-Aquaduct/PCRGlobWB20V05 already exists\n'

In [8]:
command = ("/opt/google-cloud-sdk/bin/gsutil ls %s") %(GCS_BASE)

In [9]:
keys = subprocess.check_output(command,shell=True)

In [10]:
keys = keys.decode('UTF-8').splitlines()

Removing first item from the list. The first item contains a folder without file name

In [11]:
keys2 = keys[1:]

In [12]:
df = pd.DataFrame()
i = 0
for key in keys2:
    i = i+1
    outDict = splitKey(key)
    df2 = pd.DataFrame(outDict,index=[i])
    df = df.append(df2)    

In [13]:
df.head()

,extension,fileName,identifier,month,parameter,year
1,tif,global_historical_PDomWN_month_millionm3_5min_...,000,01,global_historical_PDomWN_month_millionm3_5min_...,1960
2,tif,global_historical_PDomWN_month_millionm3_5min_...,001,02,global_historical_PDomWN_month_millionm3_5min_...,1960
3,tif,global_historical_PDomWN_month_millionm3_5min_...,002,03,global_historical_PDomWN_month_millionm3_5min_...,1960
4,tif,global_historical_PDomWN_month_millionm3_5min_...,003,04,global_historical_PDomWN_month_millionm3_5min_...,1960
5,tif,global_historical_PDomWN_month_millionm3_5min_...,004,05,global_historical_PDomWN_month_millionm3_5min_...,1960


In [14]:
df.tail()

,extension,fileName,identifier,month,parameter,year
9286,tif,global_historical_soilmoisture_month_meter_5mi...,679,08,global_historical_soilmoisture_month_meter_5mi...,2014
9287,tif,global_historical_soilmoisture_month_meter_5mi...,680,09,global_historical_soilmoisture_month_meter_5mi...,2014
9288,tif,global_historical_soilmoisture_month_meter_5mi...,681,10,global_historical_soilmoisture_month_meter_5mi...,2014
9289,tif,global_historical_soilmoisture_month_meter_5mi...,682,11,global_historical_soilmoisture_month_meter_5mi...,2014
9290,tif,global_historical_soilmoisture_month_meter_5mi...,683,12,global_historical_soilmoisture_month_meter_5mi...,2014


In [15]:
df.shape

(9290, 6)

In [16]:
parameters = df.parameter.unique()

In [17]:
print(parameters)

[u'global_historical_PDomWN_month_millionm3_5min_1960_2014'
 u'global_historical_PDomWN_year_millionm3_5min_1960_2014'
 u'global_historical_PDomWW_month_millionm3_5min_1960_2014'
 u'global_historical_PDomWW_year_millionm3_5min_1960_2014'
 u'global_historical_PIndWN_month_millionm3_5min_1960_2014'
 u'global_historical_PIndWN_year_millionm3_5min_1960_2014'
 u'global_historical_PIndWW_month_millionm3_5min_1960_2014'
 u'global_historical_PIndWW_year_millionm3_5min_1960_2014'
 u'global_historical_PIrrWN_month_millionm3_5min_1960_2014'
 u'global_historical_PIrrWN_year_millionm3_5min_1960_2014'
 u'global_historical_PIrrWW_month_millionm3_5min_1960_2014'
 u'global_historical_PIrrWW_year_millionm3_5min_1960_2014'
 u'global_historical_PLivWN_month_millionm3_5min_1960_2014'
 u'global_historical_PLivWN_year_millionm3_5min_1960_2014'
 u'global_historical_PLivWW_month_millionm3_5min_1960_2014'
 u'global_historical_PLivWW_year_millionm3_5min_1960_2014'
 u'global_historical_aqbasinwaterstress_month_di

We will store the geotiff images of each NetCDF4 file in imageCollections. The imageCollections will have the same name and content as the original NetCDF4files. 


In [18]:
for parameter in parameters:
    eeLocation = EE_BASE + "/" + parameter
    command = ("earthengine create collection %s") %eeLocation
    # Uncomment the following command if you run this script for the first time
    #subprocess.check_output(command,shell=True)
    #print(command)
    

Now that the folder and collections have been created we can start ingesting the data. It is crucial to store the relevant metadata with the images. 

In [19]:
df_parameter = pd.DataFrame()
i = 0
for parameter in parameters:
    i = i+1
    outDict_parameter = splitParameter(parameter)
    df_parameter2 = pd.DataFrame(outDict_parameter,index=[i])
    df_parameter = df_parameter.append(df_parameter2)   
    

In [20]:
df_parameter.head()

,geographic_range,indicator,parameter,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units
1,global,PDomWN,global_historical_PDomWN_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3
2,global,PDomWN,global_historical_PDomWN_year_millionm3_5min_1...,5min,historical,2014,1960,year,millionm3
3,global,PDomWW,global_historical_PDomWW_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3
4,global,PDomWW,global_historical_PDomWW_year_millionm3_5min_1...,5min,historical,2014,1960,year,millionm3
5,global,PIndWN,global_historical_PIndWN_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3


In [21]:
df_parameter.tail()

,geographic_range,indicator,parameter,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units
21,global,riverdischarge,global_historical_riverdischarge_month_m3secon...,5min,historical,2014,1960,month,m3second
22,global,riverdischarge,global_historical_riverdischarge_year_m3second...,5min,historical,2014,1960,year,m3second
23,global,runoff,global_historical_runoff_month_mmonth_5min_195...,5min,historical,2014,1958,month,mmonth
24,global,runoff,global_historical_runoff_year_myear_5min_1958_...,5min,historical,2014,1958,year,myear
25,global,soilmoisture,global_historical_soilmoisture_month_meter_5mi...,5min,historical,2014,1958,month,meter


In [22]:
df_parameter.shape

(25, 9)

In [23]:
df_complete = df.merge(df_parameter,how='left',left_on='parameter',right_on='parameter')

Adding NoData value, ingested_by and exportdescription

In [24]:
df_complete["nodata"] = -9999
df_complete["ingested_by"] ="RutgerHofste"
df_complete["exportdescription"] = df_complete["indicator"] + "_" + df_complete["temporal_resolution"]+"Y"+df_complete["year"]+"M"+df_complete["month"]

In [25]:
df_complete.head()

,extension,fileName,identifier,month,parameter,year,geographic_range,indicator,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units,nodata,ingested_by,exportdescription
0,tif,global_historical_PDomWN_month_millionm3_5min_...,000,01,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M01
1,tif,global_historical_PDomWN_month_millionm3_5min_...,001,02,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M02
2,tif,global_historical_PDomWN_month_millionm3_5min_...,002,03,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M03
3,tif,global_historical_PDomWN_month_millionm3_5min_...,003,04,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M04
4,tif,global_historical_PDomWN_month_millionm3_5min_...,004,05,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M05


In [26]:
df_complete.tail()

,extension,fileName,identifier,month,parameter,year,geographic_range,indicator,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units,nodata,ingested_by,exportdescription
9285,tif,global_historical_soilmoisture_month_meter_5mi...,679,08,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M08
9286,tif,global_historical_soilmoisture_month_meter_5mi...,680,09,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M09
9287,tif,global_historical_soilmoisture_month_meter_5mi...,681,10,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M10
9288,tif,global_historical_soilmoisture_month_meter_5mi...,682,11,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M11
9289,tif,global_historical_soilmoisture_month_meter_5mi...,683,12,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M12


In [27]:
list(df_complete.columns.values)

['extension',
 'fileName',
 'identifier',
 'month',
 'parameter',
 'year',
 'geographic_range',
 'indicator',
 'spatial_resolution',
 'temporal_range',
 'temporal_range_max',
 'temporal_range_min',
 'temporal_resolution',
 'units',
 'nodata',
 'ingested_by',
 'exportdescription']

In [30]:
def uploadEE(index,row):
    target = EE_BASE +"/"+ row.parameter + "/" + row.fileName
    source = GCS_BASE + row.fileName + "." + row.extension
    metadata = "--nodata_value=%s --time_start %s-%s-01 -p extension=%s -p filename=%s -p identifier=%s -p year=%s -p geographic_range=%s -p indicator=%s -p spatial_resolution=%s -p temporal_range=%s -p temporal_range_max=%s -p temporal_range_min=%s -p temporal_resolution=%s -p units=%s -p ingested_by=%s -p exportdescription=%s" %(row.nodata,row.year,row.month,row.extension,row.fileName,row.identifier,row.year,row.geographic_range,row.indicator,row.spatial_resolution,row.temporal_range,row.temporal_range_max,row.temporal_range_min, row.temporal_resolution, row.units, row.ingested_by, row.exportdescription)
    command = "/opt/anaconda3/bin/earthengine upload image --asset_id %s %s %s" % (target, source,metadata)
    try:
        response = subprocess.check_output(command, shell=True)
        outDict = {"command":command,"response":response,"error":0}
        df_errors2 = pd.DataFrame(outDict,index=[index])
        pass
    except:
        try:
            outDict = {"command":command,"response":response,"error":1}
        except:
            outDict = {"command":command,"response":-9999}
        df_errors2 = pd.DataFrame(outDict,index=[index])
        print("error")
    return df_errors2



In [ ]:
df_errors = pd.DataFrame()
start_time = time.time()
for index, row in df_complete.iterrows():
    elapsed_time = time.time() - start_time 
    print(index,"%.2f" %((index/8548.0)*100), "elapsed: ", str(timedelta(seconds=elapsed_time)))
    df_errors2 = uploadEE(index,row)
    df_errors = df_errors.append(df_errors2)
    
    

(0, '0.00', 'elapsed: ', '0:00:00.012130')
(1, '0.01', 'elapsed: ', '0:00:01.442132')
(2, '0.02', 'elapsed: ', '0:00:02.606816')
(3, '0.04', 'elapsed: ', '0:00:03.816287')
(4, '0.05', 'elapsed: ', '0:00:04.980202')
(5, '0.06', 'elapsed: ', '0:00:06.155003')
(6, '0.07', 'elapsed: ', '0:00:07.309283')
(7, '0.08', 'elapsed: ', '0:00:08.442458')
(8, '0.09', 'elapsed: ', '0:00:09.617328')
(9, '0.11', 'elapsed: ', '0:00:10.757391')
(10, '0.12', 'elapsed: ', '0:00:12.015491')
(11, '0.13', 'elapsed: ', '0:00:13.226300')
(12, '0.14', 'elapsed: ', '0:00:14.428539')
(13, '0.15', 'elapsed: ', '0:00:15.695246')
(14, '0.16', 'elapsed: ', '0:00:16.936033')
(15, '0.18', 'elapsed: ', '0:00:18.063282')
(16, '0.19', 'elapsed: ', '0:00:19.234435')
(17, '0.20', 'elapsed: ', '0:00:20.416357')
(18, '0.21', 'elapsed: ', '0:00:21.583643')
(19, '0.22', 'elapsed: ', '0:00:22.793715')
(20, '0.23', 'elapsed: ', '0:00:23.908967')
(21, '0.25', 'elapsed: ', '0:00:25.036050')
(22, '0.26', 'elapsed: ', '0:00:26.247714'

(185, '2.16', 'elapsed: ', '0:03:45.831343')
(186, '2.18', 'elapsed: ', '0:03:46.922072')
(187, '2.19', 'elapsed: ', '0:03:47.991843')
(188, '2.20', 'elapsed: ', '0:03:49.162332')
(189, '2.21', 'elapsed: ', '0:03:50.338605')
(190, '2.22', 'elapsed: ', '0:03:51.465543')
(191, '2.23', 'elapsed: ', '0:03:52.602319')
(192, '2.25', 'elapsed: ', '0:03:53.784525')
(193, '2.26', 'elapsed: ', '0:03:55.018508')
(194, '2.27', 'elapsed: ', '0:03:56.088998')
(195, '2.28', 'elapsed: ', '0:03:57.198106')
(196, '2.29', 'elapsed: ', '0:03:58.368086')
(197, '2.30', 'elapsed: ', '0:03:59.545212')
(198, '2.32', 'elapsed: ', '0:04:00.792572')
(199, '2.33', 'elapsed: ', '0:04:02.054067')
(200, '2.34', 'elapsed: ', '0:04:03.193201')
(201, '2.35', 'elapsed: ', '0:04:04.360057')
(202, '2.36', 'elapsed: ', '0:04:05.521603')
(203, '2.37', 'elapsed: ', '0:04:06.662887')
(204, '2.39', 'elapsed: ', '0:04:07.783256')
(205, '2.40', 'elapsed: ', '0:04:08.970049')
(206, '2.41', 'elapsed: ', '0:04:10.093355')
(207, '2.4

(368, '4.31', 'elapsed: ', '0:07:26.537960')
(369, '4.32', 'elapsed: ', '0:07:27.630547')
(370, '4.33', 'elapsed: ', '0:07:28.794948')
(371, '4.34', 'elapsed: ', '0:07:29.999826')
(372, '4.35', 'elapsed: ', '0:07:31.134382')
(373, '4.36', 'elapsed: ', '0:07:32.264391')
(374, '4.38', 'elapsed: ', '0:07:33.340269')
(375, '4.39', 'elapsed: ', '0:07:34.542902')
(376, '4.40', 'elapsed: ', '0:07:35.648335')
(377, '4.41', 'elapsed: ', '0:07:37.036933')
(378, '4.42', 'elapsed: ', '0:07:38.156094')
(379, '4.43', 'elapsed: ', '0:07:39.236006')
(380, '4.45', 'elapsed: ', '0:07:40.891491')
(381, '4.46', 'elapsed: ', '0:07:42.030798')
(382, '4.47', 'elapsed: ', '0:07:43.189106')
(383, '4.48', 'elapsed: ', '0:07:44.400879')
(384, '4.49', 'elapsed: ', '0:07:45.512351')
(385, '4.50', 'elapsed: ', '0:07:46.696253')
(386, '4.52', 'elapsed: ', '0:07:47.799867')
(387, '4.53', 'elapsed: ', '0:07:48.893019')
(388, '4.54', 'elapsed: ', '0:07:50.037887')
(389, '4.55', 'elapsed: ', '0:07:51.166215')
(390, '4.5

In [ ]:
df_errors.head()

In [ ]:
!mkdir /volumes/data/temp

In [ ]:
df_errors.to_csv("/volumes/data/temp/df_errors.csv")

In [ ]:
!aws s3 cp  /volumes/data/temp/df_errors.csv s3://wri-projects/Aqueduct30/temp/df_errors.csv